In [11]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Computers.json.gz

--2022-05-23 18:52:39--  http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Computers.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7010521 (6.7M) [application/octet-stream]
Saving to: ‘meta_Computers.json.gz.2’

meta_Computers.json 100%[===================>]   6.69M  1.70MB/s    in 3.9s    

2022-05-23 18:52:43 (1.70 MB/s) - ‘meta_Computers.json.gz.2’ saved [7010521/7010521]



In [7]:
### load the meta data

data = []
with gzip.open('meta_Computers.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

18772
{'description': ['Brand new. Still in factory-sealed plastic.'], 'title': 'Kelby Training DVD: The Adobe Photoshop CS5 7-Point System for Camera Raw By Scott Kelby', 'image': ['https://images-na.ssl-images-amazon.com/images/I/31IlLImCVJL._SS40_.jpg'], 'brand': 'Kelby Training', 'rank': ['>#654,681 in Electronics (See Top 100 in Electronics)'], 'main_cat': 'Computers', 'date': 'December 2, 2011', 'asin': '0321732960'}


In [8]:
# convert list into pandas dataframe

def list_to_pd_dataframe(df):
    df = pd.DataFrame.from_dict(data)
    df3 = df.fillna('')
    df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
    return df5

df5=list_to_pd_dataframe(df)

In [9]:
def cleandata(df5):
    df5.description=df5.description.map(lambda x: str(x))
    df5.description=df5.description.str.strip("[]'")
    df5.drop(columns=['feature', 'tech1', 'also_buy', 'price', 'also_view', 'tech2','details', 'similar_item',"main_cat"],inplace=True)
    df5['brand_cat']=df5.brand.map(lambda x : 0 if len(x) <1 else 1)
    df5['rank1']=df5['rank'].map(lambda x: str(x).strip("[]'"))
    df5["rank1"]=df5["rank1"].map(lambda x: str(x))
    df5["rank1"]=df5["rank1"].str.replace(pat='>#',repl='', regex=False)
    df5["rank1"]=df5["rank1"].map(lambda x: x.replace(",",""))
    df5["rank1"]=df5["rank1"].map(lambda x: x.split("in"))
    df5["rank1"] = df5["rank1"].map(lambda x: x[0])
    df5["title_count"] = df5["title"].map(lambda x: len(x))
    df5["desc_count"] = df5["description"].map(lambda x: len(x))
    df5["img_count"] = df5["image"].map(lambda x: len(x))
    df6=df5[df5["rank1"]!='']
    df6["rank1"]=df6["rank1"].astype("int64")
    return df6

df6=cleandata(df5)
    

/var/folders/2b/8swfrzvj1b10sb2dt02_4g6c0000gn/T/ipykernel_16764/1997018992.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6["rank1"]=df6["rank1"].astype("int64")


In [12]:
X = df6[["brand_cat","title_count","desc_count","img_count"]]
y=df6["rank1"]
         
         
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
X_train.shape,X_test.shape

model = LinearRegression().fit(X_train, y_train)

In [14]:
df6

,description,title,image,brand,rank,date,asin,brand_cat,rank1,title_count,desc_count,img_count
0,Brand new. Still in factory-sealed plastic.,Kelby Training DVD: The Adobe Photoshop CS5 7-...,[https://images-na.ssl-images-amazon.com/image...,Kelby Training,"[>#654,681 in Electronics (See Top 100 in Elec...","December 2, 2011",0321732960,1,654681,88,43,1
1,"""If you're already a Photoshop user and want a...",Kelby Training DVD: Adobe Photoshop CS5 Power ...,[https://images-na.ssl-images-amazon.com/image...,Kelby Training,"[>#830,165 in Electronics (See Top 100 in Elec...","November 7, 2011",0321735722,1,830165,72,257,1
2,School To Home Notes: Note From the Teacher!\\...,TEACHERS FRIEND NOTE FROM THE TEACHER SCHOOL T...,[https://images-na.ssl-images-amazon.com/image...,TEACHERS FRIEND,"[>#1,233,887 in Electronics (See Top 100 in El...","June 6, 2007",0439651859,1,1233887,58,257,1
3,"""Dooqi is a young brand specializing in high-e...","[3-pack]Ipad Mini Screen Protector, Dooqi Scre...",[https://images-na.ssl-images-amazon.com/image...,dooqi,"[>#454,595 in Electronics (See Top 100 in Elec...","December 5, 2014",0545729971,1,454595,201,443,5
4,Cover for Nook HD,Hd+ Cheetah Cover- Nook,,Nook,"[>#85,661 in Computers & Accessories (See top ...","September 11, 2013",0594450284,1,85661,23,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18767,,"iPad Mini 4 Case, Aceguarder New Design [Dirtp...",[https://images-na.ssl-images-amazon.com/image...,,"[>#372,769 in Electronics (See Top 100 in Elec...","July 11, 2016",B01HI72XZW,0,372769,187,0,6
18768,,MIJIU Nylon Canvas Military Tactical Men Waist...,[https://images-na.ssl-images-amazon.com/image...,,"[>#308,759 in Electronics (See Top 100 in Elec...","June 29, 2016",B01HIEI83Q,0,308759,55,0,5
18769,Deadpool is a mercenary and anti-hero appearin...,Marvel Deadpool Laptop Backpack,[https://images-na.ssl-images-amazon.com/image...,Bioworld,"[>#4,064 in Computers & Accessories (See top 1...","June 24, 2016",B01HIQYNJM,1,4064,31,297,4
18770,,Funnel Cup (Ultra Pink),[https://images-na.ssl-images-amazon.com/image...,JDDZ,"[>#449,492 in Electronics (See Top 100 in Elec...","March 5, 2017",B01HIWBL46,1,449492,23,0,1
